В данном ноутбуке попробуем подобрать гипер параметры для модели Catboost с помощью библиотеки optuna.

In [1]:
# !pip install catboost
# !pip install optuna

In [2]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time
import optuna

In [3]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [4]:
#df = pd.read_csv('/content/drive/MyDrive/data/lenta_mystem_lemmas_str_largecol_v02.csv')
df = pd.read_csv('lenta_mystem_lemmas_str_largecol_v02.csv')

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,date,url,topic,tags,text_str,topic_le,title_lemmas,year,month,day,date_enc,season,day_of_week,dummy_weekday
0,1999-08-31,https://lenta.ru/news/1999/08/31/stancia_mir/,Россия,Все,становиться известно агентство ассошиэйтед пре...,20,космонавт сомневаться надежность мир,1999,8,31,0,3,1,1
1,1999-08-31,https://lenta.ru/news/1999/08/31/vzriv/,Россия,Все,зал игровой автомат третий ярус подземный комп...,20,взрыв центр москва пострадать человек,1999,8,31,0,3,1,1
2,1999-08-31,https://lenta.ru/news/1999/08/31/credit_japs/,Россия,Все,япония принимать решение размораживать кредит ...,20,япония кредитовать россия полтора миллиард доллар,1999,8,31,0,3,1,1
3,1999-08-31,https://lenta.ru/news/1999/08/31/diana/,Мир,Все,британец отмечать сегодня скорбный дата вторую...,13,британец отмечать двухлетие смерть диана,1999,8,31,0,3,1,1
4,1999-08-31,https://lenta.ru/news/1999/08/31/mvf/,Россия,Все,понедельник директор департамент внешний связь...,20,отмывать bank деньги иметь отношение мвф,1999,8,31,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153089,2022-11-18,https://lenta.ru/news/2022/11/18/bezhh/,Мир,Общество,беженец украина избивать город кейзер американ...,13,беженец украина избивать спор нацист,2022,11,18,8472,4,4,1
1153090,2022-11-18,https://lenta.ru/news/2022/11/18/nurmagomedov/,Спорт,Бокс и ММА,российский боец смешанный стиль мма двоюродный...,23,брат нурмагомедов называть цель выступление be...,2022,11,18,8472,4,4,1
1153091,2022-11-18,https://lenta.ru/news/2022/11/18/bioprint/,Наука и техника,Будущее,ученый израильский технологический институт ра...,16,разрабатывать технология печать ткань транспла...,2022,11,18,8472,4,4,1
1153092,2022-11-18,https://lenta.ru/news/2022/11/18/penni/,Мир,Политика,республиканец палата представитель конгресс сш...,13,республиканец конгресс потребовать проверять к...,2022,11,18,8472,4,4,1


In [6]:
df.columns

Index(['date', 'url', 'topic', 'tags', 'text_str', 'topic_le', 'title_lemmas',
       'year', 'month', 'day', 'date_enc', 'season', 'day_of_week',
       'dummy_weekday'],
      dtype='object')

In [7]:
df['topic'].value_counts().index[-6:]

Index(['Библиотека', 'Оружие', 'ЧМ-2014', 'Мотор', 'МедНовости', 'Сочи'], dtype='object')

In [8]:
names = list(df['topic'].value_counts().index[-6:])
df = df[~df['topic'].isin(names)]
df['topic'].value_counts()

Россия               241055
Мир                  195778
Экономика            122120
Бывший СССР           93769
Спорт                 91267
Наука и техника       75609
Культура              71765
Интернет и СМИ        64800
Из жизни              44812
Силовые структуры     43287
Путешествия           22756
Ценности              22323
Дом                   22132
Среда обитания        15607
Бизнес                 7430
Моя страна             6128
Нацпроекты             4728
69-я параллель         3356
Забота о себе          3174
Крым                    666
Культпросвет            340
Легпром                 114
Name: topic, dtype: int64

In [9]:
dftrain, dftest, ytrain, ytest = train_test_split(df, df['topic_le'],
                                                  test_size=0.1,
                                                  stratify=df['topic'], 
                                                  random_state=123)

Xtrain, Xtest, ytrain, ytest = train_test_split(dftrain, dftrain['topic_le'],
                                                  test_size=0.2,
                                                  stratify=dftrain['topic'],
                                                  random_state=123)

X1, X, y1, y = train_test_split(Xtrain, Xtrain['topic_le'],
                                                  test_size=0.05,
                                                  stratify=Xtrain['topic'],
                                                  random_state=123)

In [10]:
del df
del dftest
del X1

In [11]:
Xtest.shape

(207543, 14)

In [12]:
# Соотнесения закодированного таргета и названий категорий
dict_topic = dict(zip(Xtest.topic, Xtest.topic_le))
dict_topic = dict(sorted(dict_topic.items(), key=lambda item: item[1]))

In [13]:
X_test = Xtest[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']]

text_cols = ['text_str', 'title_lemmas']
cat_cols = ['year', 'day_of_week', 'season']

Будем подбирать следующие гиперпараметры:
 - `n_estimators` - кол-во деревьев в катбусте
 - `depth` - максимальная глубина деревьев
 - `learning_rate` подбираться не будет, тк catboost сам неплохо его определяет при заданных других гиперпараметрах

In [14]:
def objective(trial):
    params = {
        #"learning_rate": trial.suggest_float("learning_rate", 0.001, 0.9),
        "n_estimators": trial.suggest_int("n_estimators", 800, 3000),
        "depth": trial.suggest_int("depth", 3, 10), 
        "loss_function": 'MultiClass',
        "eval_metric": 'TotalF1',
        "verbose": 300,
        "task_type": 'GPU'
    }
    
    clf = CatBoostClassifier(random_state=123, 
                             **params)
    
    clf.fit(X_test, ytest, text_features=text_cols, cat_features=cat_cols)
    preds = clf.predict(X[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']])
    
    # оцениваем качество
    f1 = f1_score(y, preds, average='micro')
    print(f'f1-score: {f1}')
    #print(classification_report(y, preds, target_names=dict_topic))
    
    return f1

In [15]:
start = time.time()

study = optuna.create_study(study_name="catboost", direction='maximize')
study.optimize(objective, n_trials=15)

print(f'время, затраченное на подбор гиперов = {time.time() - start}')

[I 2023-02-27 13:43:21,594] A new study created in memory with name: catboost


Learning rate set to 0.106558
0:	learn: 0.5788394	total: 618ms	remaining: 20m 57s
300:	learn: 0.8352138	total: 1m 53s	remaining: 10m 53s
600:	learn: 0.8547123	total: 3m 37s	remaining: 8m 39s
900:	learn: 0.8665759	total: 5m 17s	remaining: 6m 40s
1200:	learn: 0.8747240	total: 6m 53s	remaining: 4m 48s
1500:	learn: 0.8811469	total: 8m 28s	remaining: 3m 1s
1800:	learn: 0.8862100	total: 10m 2s	remaining: 1m 18s
2036:	learn: 0.8896539	total: 11m 14s	remaining: 0us


[I 2023-02-27 13:55:31,496] Trial 0 finished with value: 0.8622949239923872 and parameters: {'n_estimators': 2037, 'depth': 6}. Best is trial 0 with value: 0.8622949239923872.


f1-score: 0.8622949239923872
Learning rate set to 0.09517
0:	learn: 0.4639071	total: 494ms	remaining: 19m 10s
300:	learn: 0.8164674	total: 1m 25s	remaining: 9m 40s
600:	learn: 0.8337197	total: 2m 47s	remaining: 8m 3s
900:	learn: 0.8443616	total: 4m 8s	remaining: 6m 35s
1200:	learn: 0.8517352	total: 5m 27s	remaining: 5m 8s
1500:	learn: 0.8572024	total: 6m 45s	remaining: 3m 44s
1800:	learn: 0.8613923	total: 8m 4s	remaining: 2m 23s
2100:	learn: 0.8651104	total: 9m 21s	remaining: 1m 1s
2332:	learn: 0.8678839	total: 10m 20s	remaining: 0us


[I 2023-02-27 14:06:47,270] Trial 1 finished with value: 0.8558384928569708 and parameters: {'n_estimators': 2333, 'depth': 4}. Best is trial 0 with value: 0.8622949239923872.


...

f1-score: 0.8684863523573201
Learning rate set to 0.077594
0:	learn: 0.6803222	total: 1.12s	remaining: 55m 42s
300:	learn: 0.8492545	total: 4m 7s	remaining: 36m 45s
600:	learn: 0.8736448	total: 7m 45s	remaining: 30m 43s
900:	learn: 0.8901313	total: 11m 9s	remaining: 25m 44s
1200:	learn: 0.9021453	total: 14m 20s	remaining: 21m 15s
1500:	learn: 0.9114057	total: 17m 21s	remaining: 17m 6s
1800:	learn: 0.9185272	total: 20m 16s	remaining: 13m 16s
2100:	learn: 0.9250707	total: 23m 10s	remaining: 9m 42s
2400:	learn: 0.9305147	total: 26m	remaining: 6m 17s
2700:	learn: 0.9349565	total: 28m 46s	remaining: 2m 58s
2980:	learn: 0.9388093	total: 31m 18s	remaining: 0us


[I 2023-02-27 18:00:40,100] Trial 13 finished with value: 0.8684863523573201 and parameters: {'n_estimators': 2981, 'depth': 9}. Best is trial 11 with value: 0.8686308993230384.


f1-score: 0.8684863523573201
Learning rate set to 0.077768
0:	learn: 0.6986891	total: 1.62s	remaining: 1h 20m 20s
300:	learn: 0.8572450	total: 5m 57s	remaining: 52m 52s
600:	learn: 0.8854953	total: 11m 9s	remaining: 44m 3s
900:	learn: 0.9040248	total: 15m 51s	remaining: 36m 27s
1200:	learn: 0.9175854	total: 20m 15s	remaining: 29m 53s
1500:	learn: 0.9276035	total: 24m 23s	remaining: 23m 55s
1800:	learn: 0.9353711	total: 28m 20s	remaining: 18m 26s
2100:	learn: 0.9417250	total: 32m 14s	remaining: 13m 22s
2400:	learn: 0.9473402	total: 36m	remaining: 8m 34s
2700:	learn: 0.9514817	total: 39m 41s	remaining: 3m 59s
2972:	learn: 0.9550641	total: 43m 1s	remaining: 0us


[I 2023-02-27 18:44:39,573] Trial 14 finished with value: 0.8683418053916018 and parameters: {'n_estimators': 2973, 'depth': 10}. Best is trial 11 with value: 0.8686308993230384.


f1-score: 0.8683418053916018
время, затраченное на подбор гиперов = 18077.979670763016


In [16]:
print(f"Number of finished trials: {len(study.trials)}")
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}") # выводим налучшие параметры
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 15
Best trial:
  Value: 0.8686308993230384
  Params: 
    n_estimators: 2998
    depth: 9


**Вывод**
Как можно заметить, оптимальные гиперпараметры совпали с теми, котрый были подобраны вручную с помощью поиска по сетке и использовали в финальной версии бустинга.  

Единственное различие в том, что использовалось 1600 деревьев, а не 2998 как сказала optuna. Однако еще в поиске по сетке была видна зависимость увеличения качества алгоритма от кол-ва деревьев в нем. Однако слишком большое кол-во деверьев приводит к чересчур долгому обучениб бустинга даже на gpu. Поэтому было найдено оптимальное кол-во деревьев, после которого прирост в качестве незначителен по сравнению с ростом времени обучения.

Поэтому мы не будет обучать модель с таким набором гиперпараметров на всех данных.